### Branch and Bound  

In [10]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import math

def drawGraph(A):
    layout = nx.spring_layout(A)
    nx.draw(A, layout)
    nx.draw_networkx_edge_labels(A, pos=layout, edge_labels=nx.get_edge_attributes(A, 'weight'))
    labels = {i: chr(ord('a') + i) for i in range(len(A))}
    nx.draw_networkx_labels(A, pos=layout, labels=labels)
    plt.show()

A = [[0, 3, 1, 5, 8],
    [3, 0, 6, 7, 9],
    [1, 6, 0, 4, 2],
    [5, 7, 4, 0, 3],
    [8, 9, 2, 3, 0]]

A = nx.from_numpy_matrix(np.matrix(A), create_using=nx.Graph)
drawGraph(A)

In [11]:
def euclideanDistance(x1, y1, x2, y2):
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def manhattanDistance(x1, y1, x2, y2):
    return abs(x1 - x2) + abs(y1 - y2)

def generateInstance(n, costFunction):
    graph = nx.complete_graph(n)
    nodes = []
    for i in range(n):
        newX = np.random.randint(1, n)
        newY = np.random.randint(1, n)
        for j in range(len(nodes)):
            if j == i:
                graph[i][j]['weight'] = 0
            else:
                x = nodes[j][0]
                y = nodes[j][1]
                graph[i][j]['weight'] = costFunction(x, y, newX, newY)
                graph[j][i]['weight'] = graph[i][j]['weight']
        nodes.append((newX, newY))
    return graph

In [12]:
def printGraphAdjacencyMatrix(graph):
    print("Adjacency matrix:")
    for i in range(graph.number_of_nodes()):
        for j in range(graph.number_of_nodes()):
            if i == j:
                print(0, end="\t")
            else:
                print(graph[i][j]['weight'], end="\t")
        print()

In [13]:
class Node:
    def __init__(self, bound, boundEdges, level, cost, solution):
        self.bound = bound
        self.boundEdges = boundEdges
        self.level = level
        self.cost = cost
        self.solution = solution
    
    def __lt__(self, other):
        return self.bound < other.bound
    
    def __repr__(self) -> str:
        return f"Node({self.bound}, {self.boundEdges}, {self.level}, {self.cost}, {self.solution})"


def findTwoMinimalEdges(list):
    min1 = np.inf
    min2 = np.inf
    for j in list:
        if list[j]['weight'] < min1:
            min2 = min1
            min1 = list[j]['weight']
        elif list[j]['weight'] < min2:
            min2 = list[j]['weight']
    return min1, min2

def findInitialBound(A):
    bound = 0
    initialBoundEdges = np.zeros((A.number_of_nodes(), 2), dtype=list)
    for i in range(A.number_of_nodes()):
        min1, min2 = findTwoMinimalEdges(A[i])
        initialBoundEdges[i][0] = min1
        initialBoundEdges[i][1] = min2
        bound += min1 + min2
    return math.ceil(bound / 2), initialBoundEdges

def findBound(A, solution, boundEdges):
    changedEdges = np.zeros(A.number_of_nodes(), dtype=int)
    newEdges = np.array(boundEdges)
    edgeWeight = A[solution[-2]][solution[-1]]['weight']
    if newEdges[solution[-2]][0] != edgeWeight:
        if changedEdges[solution[-2]] == 0:
            newEdges[solution[-2]][1] = edgeWeight
        else:
            newEdges[solution[-2]][0] = edgeWeight
        changedEdges[solution[-2]] += 1
    if newEdges[solution[-1]][0] != edgeWeight:
        if changedEdges[solution[-1]] == 0:
            newEdges[solution[-1]][1] = edgeWeight
        else:
            newEdges[solution[-1]][0] = edgeWeight
        changedEdges[solution[-1]] += 1
    sum = np.sum(newEdges)
    return math.ceil(sum / 2), newEdges


In [14]:
from heapq import heappush, heappop

def branchAndBoundTSP(A):
    initialBound, initialBoundEdges = findInitialBound(A)
    root = Node(initialBound, initialBoundEdges, 1, 0, [0])
    heap = []
    heappush(heap, root)
    best = np.inf
    solution = []
    while heap:
        node = heappop(heap)
        
        if node.level > A.number_of_nodes():
            if best > node.cost:
                best = node.cost
                solution = node.solution
        else:
            if node.bound < best:
                if node.level < A.number_of_nodes():
                    for k in range(1, A.number_of_nodes()):
                        try:
                            edgeWeight = A[node.solution[-1]][k]['weight']
                        except KeyError:
                            continue
                        newBound, newEdges = findBound(A, node.solution + [k], node.boundEdges) 
                        if k not in node.solution and newBound < best:
                            newNode = Node(newBound, newEdges, node.level + 1, node.cost + edgeWeight, node.solution + [k])
                            if k == 2:
                                if 1 not in node.solution:
                                    continue 
                            heappush(heap, newNode)
                else:
                    newBound, newEdges = findBound(A, node.solution + [0], node.boundEdges)
                    edgeWeight = A[node.solution[-1]][0]['weight']
                    if edgeWeight != 0 and newBound < best:
                        newNode = Node(newBound, newEdges, node.level + 1, node.cost + edgeWeight, node.solution + [0])
                        heappush(heap, newNode)
    return best, solution

In [15]:
branchAndBoundTSP(A)

(16, [0, 1, 3, 4, 2, 0])

### Twice Around The Tree

In [16]:
G2 = [[0, 4, 8, 9, 12],
    [4, 0, 6, 8, 9],
    [8, 6, 0, 10, 11],
    [9, 8, 10, 0, 7],   
    [12, 9, 11, 7, 0]]

G2 = nx.from_numpy_matrix(np.matrix(G2), create_using=nx.Graph)

drawGraph(G2)

def findPathWeight(A, path):
    weight = 0
    for i in range(len(path) - 1):
        weight += A[path[i]][path[i + 1]]['weight']
    return weight

def twiceAroundTheTreeTSP(A):
    MST = nx.minimum_spanning_tree(A)
    path = nx.dfs_preorder_nodes(MST, 0)
    path = list(path)
    path.append(path[0])
    weight = findPathWeight(A, path)
    return weight, path

In [17]:
twiceAroundTheTreeTSP(G2)

(39, [0, 1, 2, 3, 4, 0])

### Christofides